In [1]:
#This Stack Requires:
# 1. Docker
# 2. Jupyter
# 3. Selenium
# 4. RSelenium

#Practically, I wanted to set up the basics of a set for a predictor that links the year, the content, and the 3rd party views on youtube.
#I wanted to practice selinum scraping in R, and this seemed a perfect example as this already has links
#This project made me realize how much more limited RSelenium is

#This Stack Requires:

# 1. Docker

# 2. Jupyter

# 3. Selenium

# 4. RSelenium



#Practically, I wanted to set up the basics of a set for a predictor that links the year, the content, and the 3rd party views on youtube.

#I wanted to practice selinum scraping in R, and this seemed a perfect example as this already has links

#In terms of Article, this 

In [2]:
#Start your Selinum Instance Now
#docker run -d -p 4444:4444 --shm-size="2g" selenium/standalone-chrome:4.1.2-20220131

In [29]:
Fresh_Data <- TRUE #This will refresh data (Not using caching from my git)
Fresh_Computes <- TRUE  #Uses docker to pull and augment data, requires the Selenium Instance
Dev_Checks <- FALSE #Various Extra Print statements that make debugging easier

In [4]:
library('RSelenium')

Warning message:
"package 'RSelenium' was built under R version 3.6.3"Registered S3 method overwritten by 'openssl':
  method      from
  print.bytes Rcpp


In [33]:
url.data <- "https://raw.githubusercontent.com/fivethirtyeight/superbowl-ads/main/superbowl-ads.csv"

raw <- read.csv(url(url.data), header = TRUE,) 

In [34]:
head(raw)

year,brand,superbowl_ads_dot_com_url,youtube_url,funny,show_product_quickly,patriotic,celebrity,danger,animals,use_sex
2018,Toyota,https://superbowl-ads.com/good-odds-toyota/,https://www.youtube.com/watch?v=zeBZvwYQ-hA,False,False,False,False,False,False,False
2020,Bud Light,https://superbowl-ads.com/2020-bud-light-seltzer-inside-posts-brain/,https://www.youtube.com/watch?v=nbbp0VW7z8w,True,True,False,True,True,False,False
2006,Bud Light,https://superbowl-ads.com/2006-bud-light-bear-attack/,https://www.youtube.com/watch?v=yk0MQD5YgV8,True,False,False,False,True,True,False
2018,Hynudai,https://superbowl-ads.com/hope-detector-nfl-super-bowl-lii-hyundai/,https://www.youtube.com/watch?v=lNPccrGk77A,False,True,False,False,False,False,False
2003,Bud Light,https://superbowl-ads.com/2003-bud-light-hermit-crab/,https://www.youtube.com/watch?v=ovQYgnXHooY,True,True,False,False,True,True,True
2020,Toyota,https://superbowl-ads.com/2020-toyota-go-places-with-cobie-smulders/,https://www.youtube.com/watch?v=f34Ji70u3nk,True,True,False,True,True,True,False


In [7]:
#This is the raw data from fivethirtyeight
#Now we're going to augment it

In [35]:
if (Fresh_Computes && Fresh_Data){
    remDr <- RSelenium::remoteDriver(remoteServerAddr = "localhost",
                                 port = 4444,
                                 browserName = "chrome")
    remDr$open()
}

[1] "Connecting to remote server"
$acceptInsecureCerts
[1] FALSE

$browserName
[1] "chrome"

$browserVersion
[1] "89.0.4389.90"

$chrome
$chrome$chromedriverVersion
[1] "89.0.4389.23 (61b08ee2c50024bab004e48d2b1b083cdbdac579-refs/branch-heads/4389@{#294})"

$chrome$userDataDir
[1] "/tmp/.com.google.Chrome.vCqsyT"


$`goog:chromeOptions`
$`goog:chromeOptions`$debuggerAddress
[1] "localhost:39799"


$networkConnectionEnabled
[1] FALSE

$pageLoadStrategy
[1] "normal"

$platformName
[1] "linux"

$proxy
named list()

$`se:cdp`
[1] "http://172.17.0.2:4444/session/9d0f56b5d0ab5b15b81784eede234d96/se/cdp"

$setWindowRect
[1] TRUE

$strictFileInteractability
[1] FALSE

$timeouts
$timeouts$implicit
[1] 0

$timeouts$pageLoad
[1] 300000

$timeouts$script
[1] 30000


$unhandledPromptBehavior
[1] "dismiss and notify"

$`webauthn:extension:largeBlob`
[1] TRUE

$`webauthn:virtualAuthenticators`
[1] TRUE

$id
[1] "9d0f56b5d0ab5b15b81784eede234d96"



In [9]:
var <- as.character(raw$youtube[2])
print(var)


[1] "https://www.youtube.com/watch?v=nbbp0VW7z8w"


In [15]:
if (Fresh_Computes && Fresh_Data && Dev_Checks){
    remDr$navigate("https://www.youtube.com/watch?v=lNPccrGk77A") #Entering our URL gets the browser to navigate to the page
    remDr$screenshot(display = TRUE)
}


Selenium message:no such element: Unable to locate element: {"method":"css selector","selector":"[class='view-count style-scope ytd-video-view-count-renderer']"}
  (Session info: chrome=89.0.4389.90)
For documentation on this error, please visit: https://selenium.dev/exceptions/#no_such_element
Build info: version: '4.0.0-beta-3', revision: '909239cd9f'
System info: host: '7a0166dbe9c7', ip: '172.17.0.2', os.name: 'Linux', os.arch: 'amd64', os.version: '5.4.72-microsoft-standard-WSL2', java.version: '1.8.0_282'
Driver info: driver.version: unknown



ERROR: Error: 	 Summary: NoSuchElement
 	 Detail: An element could not be located on the page using the given search parameters.
 	 class: org.openqa.selenium.NoSuchElementException
	 Further Details: run errorDetails method


In [16]:
if (Fresh_Computes && Fresh_Data && Dev_Checks){    
    webElem <- remDr$findElement(using = "css selector", "[class='view-count style-scope ytd-video-view-count-renderer']")
    view_number_text = webElem$getElementText()
    print(view_number_text[])
}

[[1]]
[1] "13,860 views"



In [17]:
if (Fresh_Computes && Fresh_Data && Dev_Checks){
    webElem <- remDr$findElement(using = "css", "[class='view-count style-scope ytd-video-view-count-renderer']")
    view_number_text = webElem$getElementText()
    print(view_number_text)
    view_number_text = gsub("views", "", view_number_text)
    view_number_text = gsub(",", "", view_number_text)
    print(view_number_text)
}

[[1]]
[1] "238 views"

[1] "238 "


In [36]:
raw$Views <- "NAN"

In [37]:
if (Dev_Checks) {
    head(raw)
}

year,brand,superbowl_ads_dot_com_url,youtube_url,funny,show_product_quickly,patriotic,celebrity,danger,animals,use_sex,Views
2018,Toyota,https://superbowl-ads.com/good-odds-toyota/,https://www.youtube.com/watch?v=zeBZvwYQ-hA,False,False,False,False,False,False,False,NAN
2020,Bud Light,https://superbowl-ads.com/2020-bud-light-seltzer-inside-posts-brain/,https://www.youtube.com/watch?v=nbbp0VW7z8w,True,True,False,True,True,False,False,NAN
2006,Bud Light,https://superbowl-ads.com/2006-bud-light-bear-attack/,https://www.youtube.com/watch?v=yk0MQD5YgV8,True,False,False,False,True,True,False,NAN
2018,Hynudai,https://superbowl-ads.com/hope-detector-nfl-super-bowl-lii-hyundai/,https://www.youtube.com/watch?v=lNPccrGk77A,False,True,False,False,False,False,False,NAN
2003,Bud Light,https://superbowl-ads.com/2003-bud-light-hermit-crab/,https://www.youtube.com/watch?v=ovQYgnXHooY,True,True,False,False,True,True,True,NAN
2020,Toyota,https://superbowl-ads.com/2020-toyota-go-places-with-cobie-smulders/,https://www.youtube.com/watch?v=f34Ji70u3nk,True,True,False,True,True,True,False,NAN


In [38]:
if (Fresh_Computes && Fresh_Data){
    length <- nrow(raw)
    if (Dev_Checks){
        print("Dev Mode Enabled")
        length <- 5
    }
    for (row_number in 1:length){
        remDr$navigate(as.character(raw$youtube[row_number]))
        Sys.sleep(5.0) #Could not find a way to wait for complete page load, so I threw in a wait to ensure load
        webElem <- tryCatch({remDr$findElement(using = "css", "[class='view-count style-scope ytd-video-view-count-renderer']")},
                            error = function(e){ print("Could Not Find Video")})
        view_number_text <- ""
        if(!is.null(webElem)){
            view_number_text <- tryCatch({webElem$getElementText()},
                                error = function(e){ print("Could Not get View Count Value")})
        } else {
            print("I could not find target")
        }
        view_number_text = gsub("views", "", view_number_text)
        view_number_text = gsub(",", "", view_number_text)
        if (Dev_Checks){
            print(raw[row_number,]['youtube_url'])
            print(view_number_text)
        }
        raw$Views[row_number] <- view_number_text
    }
    if (Dev_Checks) {
        write.csv(raw,"test_data.csv", row.names = FALSE)
    } else {
        write.csv(raw,"Superbowl_adds_count.csv", row.names = FALSE)
    }
}
else {
    url.data <- "https://raw.githubusercontent.com/fivethirtyeight/superbowl-ads/main/superbowl-ads.csv"
    raw <- read.csv(url(url.data), header = TRUE,) 
}


Selenium message:no such element: Unable to locate element: {"method":"css selector","selector":"[class='view-count style-scope ytd-video-view-count-renderer']"}
  (Session info: chrome=89.0.4389.90)
For documentation on this error, please visit: https://selenium.dev/exceptions/#no_such_element
Build info: version: '4.0.0-beta-3', revision: '909239cd9f'
System info: host: '7a0166dbe9c7', ip: '172.17.0.2', os.name: 'Linux', os.arch: 'amd64', os.version: '5.4.72-microsoft-standard-WSL2', java.version: '1.8.0_282'
Driver info: driver.version: unknown



[1] "Could Not Find Video"
[1] "Could Not get View Count Value"



Selenium message:no such element: Unable to locate element: {"method":"css selector","selector":"[class='view-count style-scope ytd-video-view-count-renderer']"}
  (Session info: chrome=89.0.4389.90)
For documentation on this error, please visit: https://selenium.dev/exceptions/#no_such_element
Build info: version: '4.0.0-beta-3', revision: '909239cd9f'
System info: host: '7a0166dbe9c7', ip: '172.17.0.2', os.name: 'Linux', os.arch: 'amd64', os.version: '5.4.72-microsoft-standard-WSL2', java.version: '1.8.0_282'
Driver info: driver.version: unknown



[1] "Could Not Find Video"
[1] "Could Not get View Count Value"



Selenium message:no such element: Unable to locate element: {"method":"css selector","selector":"[class='view-count style-scope ytd-video-view-count-renderer']"}
  (Session info: chrome=89.0.4389.90)
For documentation on this error, please visit: https://selenium.dev/exceptions/#no_such_element
Build info: version: '4.0.0-beta-3', revision: '909239cd9f'
System info: host: '7a0166dbe9c7', ip: '172.17.0.2', os.name: 'Linux', os.arch: 'amd64', os.version: '5.4.72-microsoft-standard-WSL2', java.version: '1.8.0_282'
Driver info: driver.version: unknown



...

[1] "Could Not Find Video"
[1] "Could Not get View Count Value"



Selenium message:no such element: Unable to locate element: {"method":"css selector","selector":"[class='view-count style-scope ytd-video-view-count-renderer']"}
  (Session info: chrome=89.0.4389.90)
For documentation on this error, please visit: https://selenium.dev/exceptions/#no_such_element
Build info: version: '4.0.0-beta-3', revision: '909239cd9f'
System info: host: '7a0166dbe9c7', ip: '172.17.0.2', os.name: 'Linux', os.arch: 'amd64', os.version: '5.4.72-microsoft-standard-WSL2', java.version: '1.8.0_282'
Driver info: driver.version: unknown



[1] "Could Not Find Video"
[1] "Could Not get View Count Value"


Warning message in file(file, ifelse(append, "a", "w")):
"cannot open file 'Superbowl_adds_count.csv': Permission denied"

ERROR: Error in file(file, ifelse(append, "a", "w")): cannot open the connection


In [40]:
head(raw)

year,brand,superbowl_ads_dot_com_url,youtube_url,funny,show_product_quickly,patriotic,celebrity,danger,animals,use_sex,Views
2018,Toyota,https://superbowl-ads.com/good-odds-toyota/,https://www.youtube.com/watch?v=zeBZvwYQ-hA,False,False,False,False,False,False,False,185328
2020,Bud Light,https://superbowl-ads.com/2020-bud-light-seltzer-inside-posts-brain/,https://www.youtube.com/watch?v=nbbp0VW7z8w,True,True,False,True,True,False,False,78717
2006,Bud Light,https://superbowl-ads.com/2006-bud-light-bear-attack/,https://www.youtube.com/watch?v=yk0MQD5YgV8,True,False,False,False,True,True,False,142558
2018,Hynudai,https://superbowl-ads.com/hope-detector-nfl-super-bowl-lii-hyundai/,https://www.youtube.com/watch?v=lNPccrGk77A,False,True,False,False,False,False,False,240
2003,Bud Light,https://superbowl-ads.com/2003-bud-light-hermit-crab/,https://www.youtube.com/watch?v=ovQYgnXHooY,True,True,False,False,True,True,True,13860
2020,Toyota,https://superbowl-ads.com/2020-toyota-go-places-with-cobie-smulders/,https://www.youtube.com/watch?v=f34Ji70u3nk,True,True,False,True,True,True,False,28043


In [20]:
raw<-raw[!(raw$Views=="NAN"),] #remove the empty rows
head(raw)

year,brand,superbowl_ads_dot_com_url,youtube_url,funny,show_product_quickly,patriotic,celebrity,danger,animals,use_sex,Views
2018,Toyota,https://superbowl-ads.com/good-odds-toyota/,https://www.youtube.com/watch?v=zeBZvwYQ-hA,False,False,False,False,False,False,False,185322
2020,Bud Light,https://superbowl-ads.com/2020-bud-light-seltzer-inside-posts-brain/,https://www.youtube.com/watch?v=nbbp0VW7z8w,True,True,False,True,True,False,False,78711
2006,Bud Light,https://superbowl-ads.com/2006-bud-light-bear-attack/,https://www.youtube.com/watch?v=yk0MQD5YgV8,True,False,False,False,True,True,False,142558
2018,Hynudai,https://superbowl-ads.com/hope-detector-nfl-super-bowl-lii-hyundai/,https://www.youtube.com/watch?v=lNPccrGk77A,False,True,False,False,False,False,False,238
2003,Bud Light,https://superbowl-ads.com/2003-bud-light-hermit-crab/,https://www.youtube.com/watch?v=ovQYgnXHooY,True,True,False,False,True,True,True,13860


In [26]:
c_dat = subset(raw, select = -c(superbowl_ads_dot_com_url,youtube_url) )

In [27]:
head(c_dat)

year,brand,funny,show_product_quickly,patriotic,celebrity,danger,animals,use_sex,Views
2018,Toyota,False,False,False,False,False,False,False,185322
2020,Bud Light,True,True,False,True,True,False,False,78711
2006,Bud Light,True,False,False,False,True,True,False,142558
2018,Hynudai,False,True,False,False,False,False,False,238
2003,Bud Light,True,True,False,False,True,True,True,13860


In [24]:
unique(c_dat['brand'])

,brand
1,Toyota
2,Bud Light
4,Hynudai
7,Coca-Cola
8,Kia
10,Budweiser
15,NFL
18,Pepsi
20,Doritos
34,E-Trade


In [ ]:
#Findings and Recommendations
#Practically I wanted to develop the data backing a predictor, and I think I could probably do a better job of augmenting
# This mainly relies on scraping youtube data, where not all cases actually even have a presence. It may be better
# to look at the viewership numbers from the superbowl itself. 
# In terms of the origin data set, I would love to see a breakdown on time or % of ad as I think that would be interesting!
